In [2]:
import numpy as np
import os
import subprocess
import json
import calendar
import pandas as pd
import geopandas as gpd
import shapely
from scipy.spatial.distance import cdist

In [42]:
# testing
result = subprocess.run(["curl", 'http://localhost:5000/route/v1/driving/13.388860,52.517037;13.397634,52.529407?overview=false'], stdout=subprocess.PIPE)
resultStr = result.stdout.decode("utf-8")
print(resultStr)
resultDict = json.loads(resultStr)
distance = resultDict['routes'][0]['distance']
print(distance)

result = subprocess.run(["curl", 'http://router.project-osrm.org/route/v1/driving/13.388860,52.517037;13.397634,52.529407?overview=false'], stdout=subprocess.PIPE)
resultStr = result.stdout.decode("utf-8")
print(resultStr)
resultDict = json.loads(resultStr)
distance = resultDict['routes'][0]['distance']
print(distance)


{"waypoints":[{"location":[-86.83958,45.395992],"name":"Old Camp Road","hint":"vIoBgIWSAYA-AAAAAAAAAOMDAAAAAAAAPgAAAAAAAADjAwAAAAAAAEkBAIDk7tL6GLC0AjxMzACtWCEDCgB_BXInndc="},{"location":[-86.83958,45.395992],"name":"Old Camp Road","hint":"vIoBgIWSAYA-AAAAAAAAAOMDAAAAAAAAPgAAAAAAAADjAwAAAAAAAEkBAIDk7tL6GLC0AoJuzAD_iCEDCgB_BXInndc="}],"routes":[{"distance":0,"duration":0,"weight":0,"weight_name":"routability","legs":[{"distance":0,"duration":0,"weight":0,"summary":"","steps":[]}]}],"code":"Ok"}
0
{"routes":[{"legs":[{"summary":"","weight":452.9,"duration":387.7,"steps":[],"distance":1879.4}],"weight_name":"routability","weight":452.9,"duration":387.7,"distance":1879.4}],"waypoints":[{"hint":"3fcJgB3kRoYbAAAALQAAAAoAAAAAAAAAGwAAAC0AAAAKAAAAAAAAAAzZAAAATMwAqVghAzxMzACtWCEDAQDfCu7M9aQ=","name":"Friedrichstraße","location":[13.3888,52.517033]},{"hint":"lBsUgPh77YEKAAAACAAAABsBAAAAAAAACgAAAAgAAAAbAQAAAAAAAAzZAAB_bswAGIkhA4JuzAD_iCEDAgCfEO7M9aQ=","name":"Torstraße","location":[13.397631,52.529

In [40]:
# testing
dist = distance_street(13.388860,52.517037,13.397634,52.529407)
print(dist)
dist = distance_street(43.090407,-87.929521,43.095108,-87.947162)
print(dist)
dist = distance_street(-87.929521,43.090407,-87.947162,43.095108)
print(dist)
dist = distance_street(43.090407754371284,-87.92952162479058,43.09510847013696,-87.94716283236194)
print(dist)

http://localhost:5000/route/v1/driving/52.517037,13.38886;52.529407,13.397634?overview=false
{'waypoints': [{'location': [-87.804176, 42.490471], 'name': '', 'hint': 'z2YOgPD9EICFAQAAAAAAAAAAAAAAAAAAwwAAAAAAAAAAAAAAAAAAAIsEAIDwNsT6Z1qIAq1YIQM8TMwAAADPCnInndc='}, {'location': [-87.804176, 42.490471], 'name': '', 'hint': 'z2YOgPD9EICFAQAAAAAAAAAAAAAAAAAAwwAAAAAAAAAAAAAAAAAAAIsEAIDwNsT6Z1qIAv-IIQOCbswAAADPCnInndc='}], 'routes': [{'distance': 0, 'duration': 0, 'weight': 0, 'weight_name': 'routability', 'legs': [{'distance': 0, 'duration': 0, 'weight': 0, 'summary': '', 'steps': []}]}], 'code': 'Ok'}
0
http://localhost:5000/route/v1/driving/-87.929521,43.090407;-87.947162,43.095108?overview=false
{'waypoints': [{'location': [-87.929531, 43.090407], 'name': 'North 15th Street', 'hint': '-2gDgJXAE4CQAAAAPwAAAAAAAAAAAAAAkAAAAD8AAAAAAAAAAAAAABkBAABFTcL654GRAk9NwvrngZECAABfC3Inndc='}, {'location': [-87.947163, 43.095168], 'name': 'West Roosevelt Drive', 'hint': 'V9sCgOycE4B0AAAAJAAAAAAAAAAAAAAAd

In [41]:
# just for debugging
def distance_street(lat_1, lon_1, lat_2, lon_2):

    query = 'http://localhost:5000/route/v1/driving/%s,%s;%s,%s?overview=false' % (str(lon_1), str(lat_1), str(lon_2), str(lat_2))
    print(query)
    result = subprocess.run(["curl", query], stdout=subprocess.PIPE)
    resultStr = result.stdout.decode("utf-8")
    resultDict = json.loads(resultStr)
    print(resultDict)
    streetDistance = resultDict['routes'][0]['distance']
    #print(distance)

    return streetDistance

In [9]:
class KMeans(object):
    def __init__(self, k=8, distance = 'euclid'):
        self.k = k
        if (distance == 'euclid'):
            self._distance = 'euclidean'
        elif (distance == 'haversine'):
            self._distance = self._distance_haversine
        elif (distance == 'manhattan'):
            self._distance = self._distance_manhattan
        elif (distance == 'street'):
            self._distance = self._distance_street
        else:
            raise Exception('Invalid distance metric')
    
    def _step(self):
        """Compute distance, assign groups, recompute centers"""
        distance = cdist(self.X,self.cluster_centers,metric=self._distance)
        self.labels = distance.argmin(1)
       # centers = np.zeros((self.k,2))
        for cluster in range(self.k):
            points = self.X[self.labels == cluster]
            if len(points) == 0:
                distance = cdist(self.X,np.delete(self.cluster_centers,cluster,0),metric=self._distance)
                mean_dist = np.mean(distance,0)
                self.cluster_centers[cluster] = mean_dist.argmax()
            else:
                self.cluster_centers[cluster] = np.mean(points,0)
       # self.cluster_centers = centers
        
    def _distance_haversine(self,a,b):
        lat_1, lon_1, lat_2, lon_2 = map(np.radians,[a[0],a[1],b[0],b[1]])
        d_lat = lat_2 - lat_1
        d_lon = lon_2 - lon_1
        
        arc = np.sin(d_lat/2.0)**2 + np.cos(lat_1)*np.cos(lat_2)*np.sin(d_lon/2)**2
        
        c = 2 * np.arcsin(np.sqrt(arc))
        km = 6372.8 * c
        return km
    
    def _distance_manhattan(self, a, b):
        lat_1, lon_1, lat_2, lon_2 = map(np.radians,[a[0],a[1],b[0],b[1]])
        manhattanDistance = abs(lat_2 - lat_1) + abs(lon_2 - lon_1)
        return manhattanDistance
    
    def _distance_street(self, a, b):
        # TODO implement street view - for now just manhattan
        lat_1, lon_1, lat_2, lon_2 = map(np.radians,[a[0],a[1],b[0],b[1]])
        #manhattanDistance = abs(lat_2 - lat_1) + abs(lon_2 - lon_1)
        
        query = 'http://localhost:5000/route/v1/driving/%s,%s;%s,%s?overview=false' % (str(lon_1), str(loat_1), str(lon_2), str(lat_2))
        result = subprocess.run(["curl", query], stdout=subprocess.PIPE)
        resultStr = result.stdout.decode("utf-8")
        resultDict = json.loads(resultStr)
        streetDistance = resultDict['routes'][0]['distance']
        #print(distance)
        
        return streetDistance
   
    
    def _init_centers(self, X):
        unique = np.unique(X, axis=0)
        index = np.random.permutation(len(unique))[:self.k]
        return unique[index]
    
    def fit(self,X, centers = None):
        '''Expects centers to be inputted, if not random'''
        self.labels = np.zeros(len(X))
        self.X = X
        if centers is not None:
            self.cluster_centers = centers 
        else:
            self.cluster_centers = self._init_centers(X)
        old_centers = np.zeros((self.k,2))
    #    self.i = 0
        while(not np.array_equal(old_centers, self.cluster_centers)):
            old_centers = self.cluster_centers.copy()
            self._step()
         #   self.i+=1

In [10]:
from shapely.geometry import Point
from geopandas import GeoDataFrame

demographics = gpd.read_file('./census.geoJSON')

def gen_coords(loc):
    data = loc[1:-1].split(',')
    data = list((np.float(data[0]), np.float(data[1])))
    x.append(data[1])
    y.append(data[0])
    return [data[0],data[1]]

def point_similarity(X,geo_labels, euc_labels,k):
    '''For an inputted series of points, geodesic labels, euclidean labels, and k-value
       returns the point-similarity index per geodesic cluster
    '''

    euc_cluster_totals = np.zeros(k,dtype=np.int)
    geo_euc_composition = [np.zeros(k,dtype=np.int)* 1 for i in range(k)]
    
    for index,point in enumerate(geo_labels):
        euc_cluster_totals[euc_labels[index]] += 1
        geo_euc_composition[point][euc_labels[index]] += 1
    
    point_sim = []
    for geo_cluster in range(k):
        sim = 0
        for euc_cluster in range(k):
            matching_points = geo_euc_composition[geo_cluster][euc_cluster]
            euc_percentage = matching_points / euc_cluster_totals[euc_cluster]
            geo_percentage = matching_points / np.sum(geo_euc_composition[geo_cluster])
            sim += euc_percentage * geo_percentage
        point_sim.append(sim)

    return np.array(point_sim)

def minority_probability(X,cluster_number,geo_labels,demographics):
        points = X[geo_labels == cluster_number]
        # geoJSON puts points in Long/Lat order
        # but points are in lat/long earlier
        hull = shapely.geometry.multipoint.MultiPoint([[p[1],p[0]] for p in points]).convex_hull
  
        pop = np.zeros(7)
        for index in range(len(demographics)):
            census_tract = demographics.loc[index,'geometry']
            intersect = hull.intersection(census_tract)
            overlap = intersect.area/census_tract.area
            if (overlap != 0):
                pop = pop + (np.array(demographics.loc[index,['White','Black or African American', 'American Indian and Ala Native',
                   'Asian','Native Hawaiian/other Pac Isl', 'Multiple Race',
                   'Other Race']]) * overlap)
        
        if (np.all(pop ==0)):
            return 0
        
        return (pop[1:]/np.sum(pop)).sum()

def bias_index(X, geo_labels, euc_labels, demographics, k):
    if np.all(geo_labels == euc_labels):
        return 0

    dissimilarity_index = 1 - point_similarity(X,geo_labels,euc_labels,k)
    minority_prob = np.array([minority_probability(X,cluster,geo_labels,demographics) 
                              for cluster in range(k)])
    
    potential_bias = minority_prob * dissimilarity_index
    return potential_bias.mean()


In [11]:
columns = ['year', 'k']
for crime in ['theft', 'motor', 'assault', 'robbery']:
    for month in calendar.month_abbr:
        if month != "":
            columns.append(crime + "_" + month.lower())

print(len(columns))

frame_list = []
for year in range(2005,2017):
    for k in range(2,11):
        year_list = [str(year)]
        year_list.append(k)
        year_list.extend([np.nan for i in range(48)])
        #for _ in range(48):
        #    year_list.append(np.nan) 
        frame_list.append(year_list)
    
bias_frame = pd.DataFrame(data=frame_list, columns=columns)

bias_frame.head(12)

50


,year,k,theft_jan,theft_feb,theft_mar,theft_apr,theft_may,theft_jun,theft_jul,theft_aug,...,robbery_mar,robbery_apr,robbery_may,robbery_jun,robbery_jul,robbery_aug,robbery_sep,robbery_oct,robbery_nov,robbery_dec
0,2005,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2005,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2005,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2005,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2005,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2005,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2005,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2005,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2005,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2006,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
def store_bias(folder,file,bias_value,k):
    year_string = folder.split('data_')[1]
    year_array = np.array(bias_frame.year == year_string) 
    k_array = np.array(bias_frame.k == k)
    index = np.logical_and(year_array,k_array)
    month_string = file.split('.csv')[0]
    prefix,month = month_string.split('theft_')
    if len(prefix) == 0:
        month_index = 't_' + month
    else:
        month_index = prefix + month
    bias_frame.loc[index,month_index] = bias_value

In [13]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from geopandas import GeoDataFrame
import os
import warnings

def gen_coords(loc):
    data = loc[1:-1].split(',')
    data = list((np.float(data[0]), np.float(data[1])))
    x.append(data[1])
    y.append(data[0])
    return [data[0],data[1]]

def percent_similarity(a,b):
    return len(a[a==b])/len(a)

for year in range(2005,2017):
    folder = 'data_' + str(year)
    for file in os.listdir('../data/' + folder):
        if(file.endswith('.csv')):
            df = pd.read_csv('../data/' + folder +'/' + file, sep =';')
        
            x = []
            y = []

            df['Points'] = df['Location'].apply(gen_coords)
            points = [Point(xy) for xy in zip(x,y)]
            crs = {'init': 'epsg:4326'}
            geo_df = GeoDataFrame(df,crs=crs, geometry=points)
            theft_both = geo_df.copy()
            test_list = []

            for index in range(len(theft_both)):
                test_list.append(df.loc[index, 'Points'])

            X = np.array(test_list)

            for k in range(2,11):
                euclid = KMeans(k = k, distance = 'euclid')
                geodesic = KMeans(k = k, distance = 'haversine')
                manhattan = KMeans(k = k, distance = 'manhattan')
                #street = KMeans(k = k, distance = 'street')
                
                centers = geodesic._init_centers(X)

                euclid.fit(X, centers = centers)
                geodesic.fit(X, centers = centers) 
                manhattan.fit(X, centers = centers)
                #street.fit(X, centers = centers)

                bias_GeodesicEuclid = bias_index(X, geodesic.labels, euclid.labels, demographics, k)
                
                bias_ManhattanGeodesic = bias_index(X, manhattan.labels, geodesic.labels, demographics, k)
                bias_ManhattanEuclid = bias_index(X, manhattan.labels, euclid.labels, demographics, k)
                
                #bias_StreetEuclid = bias_index(X, street.labels, euclid.labels, demographics, k)
                #bias_StreetGeodesic = bias_index(X, street.labels, geodesic.labels, demographics, k)
                #bias_StreetManhattan = bias_index(X, street.labels, manhattan.labels, demographics, k)
                
                # print(folder,file,bias_val,k)
                # TODO FIX this function
                store_bias(folder,file,bias_GeodesicEuclid,k)
                store_bias(folder,file,bias_ManhattanGeodesic,k)
                store_bias(folder,file,bias_ManhattanEuclid,k)
                
                theft_both.loc[:,'e_cluster' + 'K' + str(k)] = euclid.labels.copy()
                theft_both.loc[:,'g_cluster' + 'K' + str(k)] = geodesic.labels.copy()
                theft_both.loc[:,'m_cluster' + 'K' + str(k)] = manhattan.labels.copy()
                #theft_both.loc[:,'s_cluster' + 'K' + str(k)] = street.labels.copy()
                


              #  print(percent_similarity(euclid.labels, geodesic.labels))



            theft_both = theft_both.drop('Points', axis=1)

            try:
                os.remove('./datamound/'+ folder + '-' + file.split('.csv')[0] + '.js')
            except FileNotFoundError:
                pass

            theft_both.to_file('./datamound/'+ folder + '-' +file.split('.csv')[0] + '.js', driver='GeoJSON')
    #         with open('./datamound/'+file.split('.csv')[0] + '.js', 'r') as original: data = original.read()
    #         with open('./datamound/'+file.split('.csv')[0] + '.js', 'w') as modified: modified.write('var both =' 
    #                                                         + data +';')
            print('./datamound/'+ folder + '-' +file.split('.csv')[0] + '.js')
          #  print('-------')
        


./datamound/data_2005-m_theft_april.js
./datamound/data_2005-m_theft_aug.js
./datamound/data_2005-m_theft_dec.js
./datamound/data_2005-m_theft_feb.js
./datamound/data_2005-m_theft_jan.js
./datamound/data_2005-m_theft_july.js
./datamound/data_2005-m_theft_june.js
./datamound/data_2005-m_theft_mar.js
./datamound/data_2005-m_theft_may.js
./datamound/data_2005-m_theft_nov.js
./datamound/data_2005-m_theft_oct.js
./datamound/data_2005-m_theft_sep.js
./datamound/data_2005-theft_april.js
./datamound/data_2005-theft_aug.js
./datamound/data_2005-theft_dec.js
./datamound/data_2005-theft_feb.js
./datamound/data_2005-theft_jan.js
./datamound/data_2005-theft_july.js
./datamound/data_2005-theft_june.js
./datamound/data_2005-theft_mar.js
./datamound/data_2005-theft_may.js
./datamound/data_2005-theft_nov.js
./datamound/data_2005-theft_oct.js
./datamound/data_2005-theft_sep.js
./datamound/data_2006-m_theft_april.js
./datamound/data_2006-m_theft_aug.js
./datamound/data_2006-m_theft_dec.js
./datamound/dat

./datamound/data_2014-m_theft_oct.js
./datamound/data_2014-m_theft_sep.js
./datamound/data_2014-theft_april.js
./datamound/data_2014-theft_aug.js
./datamound/data_2014-theft_dec.js
./datamound/data_2014-theft_feb.js
./datamound/data_2014-theft_jan.js
./datamound/data_2014-theft_july.js
./datamound/data_2014-theft_june.js
./datamound/data_2014-theft_mar.js
./datamound/data_2014-theft_may.js
./datamound/data_2014-theft_nov.js
./datamound/data_2014-theft_oct.js
./datamound/data_2014-theft_sep.js
./datamound/data_2015-m_theft_april.js
./datamound/data_2015-m_theft_aug.js
./datamound/data_2015-m_theft_dec.js
./datamound/data_2015-m_theft_feb.js
./datamound/data_2015-m_theft_jan.js
./datamound/data_2015-m_theft_july.js
./datamound/data_2015-m_theft_june.js
./datamound/data_2015-m_theft_mar.js
./datamound/data_2015-m_theft_may.js
./datamound/data_2015-m_theft_nov.js
./datamound/data_2015-m_theft_oct.js
./datamound/data_2015-m_theft_sep.js
./datamound/data_2015-theft_april.js
./datamound/data_

In [14]:
bias_frame.head()

,year,k,theft_jan,theft_feb,theft_mar,theft_apr,theft_may,theft_jun,theft_jul,theft_aug,...,t_dec,t_feb,t_jan,t_july,t_june,t_mar,t_may,t_nov,t_oct,t_sep
0,2005,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.014660,0.027080,0.024786,0.015022,0.011806,0.018499,0.011301,0.057763,0.017860,0.048108
1,2005,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.045573,0.018933,0.013175,0.204197,0.035417,0.020290,0.036622,0.056596,0.051433,0.031905
2,2005,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.044254,0.031604,0.055809,0.044377,0.068649,0.048265,0.062274,0.055272,0.033631,0.047422
3,2005,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.038673,0.239521,0.085087,0.026837,0.135777,0.028236,0.070390,0.057903,0.124983,0.091330
4,2005,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.071470,0.100025,0.071931,0.180077,0.197895,0.049302,0.065125,0.060498,0.024664,0.042111


In [16]:
files = os.listdir('./datamound/')
#print(files)
for file in files:
    nameParts = file.split("_")
    month = nameParts[-1].replace(".js", "")
    month = month[:3]
    newName = file.split(month)[0] + month + ".js"
    print(newName)
    os.replace('./datamound/'+file, './datamound/'+newName)

data_2005-m_theft_apr.js
data_2005-m_theft_apr.js
data_2005-m_theft_april_bu.js
data_2005-m_theft_aug.js
data_2005-m_theft_dec.js
data_2005-m_theft_feb.js
data_2005-m_theft_jan.js
data_2005-m_theft_jul.js
data_2005-m_theft_jul.js
data_2005-m_theft_jun.js
data_2005-m_theft_jun.js
data_2005-m_theft_mar.js
data_2005-m_theft_may.js
data_2005-m_theft_nov.js
data_2005-m_theft_oct.js
data_2005-m_theft_sep.js
data_2005-robbery_apr.js
data_2005-robbery_aug.js
data_2005-robbery_dec.js
data_2005-robbery_feb.js
data_2005-robbery_jan.js
data_2005-robbery_jul.js
data_2005-robbery_jun.js
data_2005-robbery_mar.js
data_2005-robbery_may.js
data_2005-robbery_nov.js
data_2005-robbery_oct.js
data_2005-robbery_sep.js
data_2005-simpleassault_apr.js
data_2005-simpleassault_aug.js
data_2005-simpleassault_dec.js
data_2005-simpleassault_feb.js
data_2005-simpleassault_jan.js
data_2005-simpleassault_jul.js
data_2005-simpleassault_jun.js
data_2005-simpleassault_mar.js
data_2005-simpleassault_may.js
data_2005-simple

data_2012-m_theft_aug.js
data_2012-m_theft_dec.js
data_2012-m_theft_feb.js
data_2012-m_theft_jan.js
data_2012-m_theft_jul.js
data_2012-m_theft_jul.js
data_2012-m_theft_jun.js
data_2012-m_theft_jun.js
data_2012-m_theft_mar.js
data_2012-m_theft_may.js
data_2012-m_theft_nov.js
data_2012-m_theft_oct.js
data_2012-m_theft_sep.js
data_2012-robbery_apr.js
data_2012-robbery_aug.js
data_2012-robbery_dec.js
data_2012-robbery_feb.js
data_2012-robbery_jan.js
data_2012-robbery_jul.js
data_2012-robbery_jun.js
data_2012-robbery_mar.js
data_2012-robbery_may.js
data_2012-robbery_nov.js
data_2012-robbery_oct.js
data_2012-robbery_sep.js
data_2012-simpleassault_apr.js
data_2012-simpleassault_aug.js
data_2012-simpleassault_dec.js
data_2012-simpleassault_feb.js
data_2012-simpleassault_jan.js
data_2012-simpleassault_jul.js
data_2012-simpleassault_jun.js
data_2012-simpleassault_mar.js
data_2012-simpleassault_may.js
data_2012-simpleassault_nov.js
data_2012-simpleassault_oct.js
data_2012-simpleassault_sep.js
dat

In [18]:
ordered_names = []
for crime in ['theft', 'm_theft', 'simpleassault', 'robbery']:
    for month in calendar.month_abbr:
        if month != "":
            ordered_names.append(crime + "_" + month.lower() + ".js")

data = 'var dataA = ['
for year in range(2005,2008):

    year_string = 'data_'+str(year)+'-'

    for file in ordered_names:
        reader = open('./datamound/'+ year_string + file,'r')
        data += (reader.read() + ',')
        reader.close()
        print(file)

    writer = open('halfA.js','w')
    writer.write(data + '];')
    writer.close()

data = 'var dataB = ['
for year in range(2008,2011):

    year_string = 'data_'+str(year)+'-'

    for file in ordered_names:
        reader = open('./datamound/'+ year_string + file,'r')
        data += (reader.read() + ',')
        reader.close()
        print(file)
        
    writer = open('halfB.js','w')
    writer.write(data + '];')
    writer.close()
    
data = 'var dataC = ['
for year in range(2011,2014):

    year_string = 'data_'+str(year)+'-'

    for file in ordered_names:
        reader = open('./datamound/'+ year_string + file,'r')
        data += (reader.read() + ',')
        reader.close()
        print(file)
        
    writer = open('halfC.js','w')
    writer.write(data + '];')
    writer.close()
    
    
data = 'var dataD = ['
for year in range(2014,2017):

    year_string = 'data_'+str(year)+'-'

    for file in ordered_names:
        reader = open('./datamound/'+ year_string + file,'r')
        data += (reader.read() + ',')
        reader.close()
        print(file)
        
    writer = open('halfD.js','w')
    writer.write(data + '];')
    writer.close()


theft_jan.js
theft_feb.js
theft_mar.js
theft_apr.js
theft_may.js
theft_jun.js
theft_jul.js
theft_aug.js
theft_sep.js
theft_oct.js
theft_nov.js
theft_dec.js
m_theft_jan.js
m_theft_feb.js
m_theft_mar.js
m_theft_apr.js
m_theft_may.js
m_theft_jun.js
m_theft_jul.js
m_theft_aug.js
m_theft_sep.js
m_theft_oct.js
m_theft_nov.js
m_theft_dec.js
simpleassault_jan.js
simpleassault_feb.js
simpleassault_mar.js
simpleassault_apr.js
simpleassault_may.js
simpleassault_jun.js
simpleassault_jul.js
simpleassault_aug.js
simpleassault_sep.js
simpleassault_oct.js
simpleassault_nov.js
simpleassault_dec.js
robbery_jan.js
robbery_feb.js
robbery_mar.js
robbery_apr.js
robbery_may.js
robbery_jun.js
robbery_jul.js
robbery_aug.js
robbery_sep.js
robbery_oct.js
robbery_nov.js
robbery_dec.js
theft_jan.js
theft_feb.js
theft_mar.js
theft_apr.js
theft_may.js
theft_jun.js
theft_jul.js
theft_aug.js
theft_sep.js
theft_oct.js
theft_nov.js
theft_dec.js
m_theft_jan.js
m_theft_feb.js
m_theft_mar.js
m_theft_apr.js
m_theft_may.js
m

robbery_jan.js
robbery_feb.js
robbery_mar.js
robbery_apr.js
robbery_may.js
robbery_jun.js
robbery_jul.js
robbery_aug.js
robbery_sep.js
robbery_oct.js
robbery_nov.js
robbery_dec.js
theft_jan.js
theft_feb.js
theft_mar.js
theft_apr.js
theft_may.js
theft_jun.js
theft_jul.js
theft_aug.js
theft_sep.js
theft_oct.js
theft_nov.js
theft_dec.js
m_theft_jan.js
m_theft_feb.js
m_theft_mar.js
m_theft_apr.js
m_theft_may.js
m_theft_jun.js
m_theft_jul.js
m_theft_aug.js
m_theft_sep.js
m_theft_oct.js
m_theft_nov.js
m_theft_dec.js
simpleassault_jan.js
simpleassault_feb.js
simpleassault_mar.js
simpleassault_apr.js
simpleassault_may.js
simpleassault_jun.js
simpleassault_jul.js
simpleassault_aug.js
simpleassault_sep.js
simpleassault_oct.js
simpleassault_nov.js
simpleassault_dec.js
robbery_jan.js
robbery_feb.js
robbery_mar.js
robbery_apr.js
robbery_may.js
robbery_jun.js
robbery_jul.js
robbery_aug.js
robbery_sep.js
robbery_oct.js
robbery_nov.js
robbery_dec.js


In [11]:
bias_frame.to_json('bias.js',orient='records')
with open('bias.js','r') as reader:
    data = reader.read()
with open('bias.js','w') as w:
    w.write('var bias_data =' + data + ';')
    